# Imports

In [1]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

import re

import json

# Load'n'Cook

In [3]:
DEBUG_URL = 'https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [4]:
urls_actors = []

for i in range(0, 50):
    urls_actors.append('https://www.imdb.com' + selector.xpath('.//*[@class="lister-list"]'
               '/div[@class="lister-item mode-detail"]'
              '/div[@class="lister-item-content"]/h3/a/@href')[i].extract())

In [9]:
# получение селекторов
def get_selectors(urls_actors):
    
    selectors = []
    DEBUG_URLS = urls_actors

    for url in range (0,len(urls_actors)):
        req_url = requests.get(DEBUG_URLS[url])
        response_url = HtmlResponse(url=DEBUG_URLS[url], body=req_url.content)
    
        selectors.append(Selector(response=response_url))
        
    return selectors    

In [130]:
selectors_test = get_selectors(urls_actors)

In [131]:
def xpath_selectors(selectors, urls_actors):
    xpath_lists_bio = []
    xpath_lists_born = []
    xpath_lists_movies = []
    xpath_lists_name = []
    xpath_lists_url = []
    
    
    for s in range(len(selectors)):
        xpath_lists_bio.append(selectors[s].xpath('.//*[@class="name-trivia-bio-text"]/div//text()').extract())
        xpath_lists_born.append(selectors[s].xpath('.//*[@id="name-born-info"]/time/@datetime').extract())
        
        if len(selectors_test[0].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/text()'
                             ).extract())>15:
            xpath_lists_movies.append(selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/text()'
                             ).extract()[:15])
        else:
            xpath_lists_movies.append(selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/text()'
                             ).extract())
            
        xpath_lists_name.append(selectors[s].xpath('.//*[@class="itemprop"]/text()')[0].extract())
        
        xpath_lists_url.append(urls_actors[s]+'/')
    
    return xpath_lists_bio,xpath_lists_born,xpath_lists_movies,xpath_lists_name,xpath_lists_url

In [132]:
xpath_lists_bio,xpath_lists_born,xpath_lists_movies,xpath_lists_name,xpath_lists_url = xpath_selectors(selectors_test,urls_actors)

In [117]:
def clean_bio(xpaths_selectors):
    
    bios_clean = []
    
    for j in range(0, len(xpaths_selectors)):
        test_list_new = []
        for i in range(0, len(xpaths_selectors[j])):
            test_list_new.append(xpaths_selectors[j][i].replace("\n", ""))
            test_list_new = [ele for ele in test_list_new if ele.strip() and ele!='See full bio']
            test_list_new_res = test_list_new[:-1]
            test_list_new_res = ''.join(test_list_new_res)
            test_list_new_res = re.sub(' +', ' ',test_list_new_res)
            test_list_new_res = "".join(test_list_new_res.rstrip())
            
        bios_clean.append(test_list_new_res)
        
    return bios_clean

In [199]:
clean_bios = clean_bio(xpath_lists_bio)

In [186]:
def make_dict(clean_bios):
    all_dictionaries = []
    for n in range(len(clean_bios)):
        all_dictionaries.append({'bio': clean_bios[n], 'born': ''.join(map(str, xpath_lists_born[n])), 'movies': xpath_lists_movies[n], 'name': xpath_lists_name[n],
                                'url':xpath_lists_url[n]})
        
        if all_dictionaries[n]['born']=='':
            all_dictionaries[n]['born']=None
        
        
    return all_dictionaries

In [200]:
dict_first_5 = make_dict(clean_bios)

In [209]:
with open('test_2.jl', 'w') as f:
    f.write('\n'.join(map(json.dumps, dict_first_5)))

# Поиск топ-15 популярных фильмов

In [51]:
def get_movies_urls(selectors):
    
    xpath_lists_urls_movies = []
    
    for s in range(len(selectors)):  
        
        if len(selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/@href'
                             ).extract())>15:
            for i in range(15):
                xpath_lists_urls_movies.append('https://www.imdb.com' + selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/@href'
                             )[i].extract())
        else:
            for i in range(len(selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/@href'
                             ).extract())):
                xpath_lists_urls_movies.append('https://www.imdb.com' + selectors[s].xpath('.//*[@class="filmo-category-section"]'
                             '/div[@class="filmo-row odd" or "filmo-row even"]/b/a/@href'
                             )[i].extract())
        
    return xpath_lists_urls_movies

In [74]:
def get_selectors_movies(urls_movies):
    
    selectors = []
    DEBUG_URLS = urls_movies

    for url in range (0,len(urls_movies)):
        req_url = requests.get(DEBUG_URLS[url])
        response_url = HtmlResponse(url=DEBUG_URLS[url], body=req_url.content)
    
        selectors.append(Selector(response=response_url))
        
    return selectors    

In [123]:
selectors_movies = get_selectors_movies(xpath_lists_urls_movies)

In [105]:
def get_movies_cast(selectors):
    
    xpath_lists_movies_cast = []
    xpath_lists_movies_cast_matches = []
    
    for s in range(len(selectors)): 
        
        xpath_lists_movies_cast.append(selectors[s].xpath('.//*[@data-testid="title-cast-item__actor"]/text()').extract())
        
        xpath_lists_movies_cast_matches.append([x for x in xpath_lists_movies_cast[s] if x in xpath_lists_name])
        
    return xpath_lists_movies_cast_matches

    


In [134]:
xpath_lists_movies_cast_matches = get_movies_cast(selectors_movies)

In [135]:
xpath_lists_urls_movies = get_movies_urls(selectors_test)

In [144]:
def make_dict_movies():
    all_dictionaries = []
    for n in range(len(xpath_lists_movies)):
        for t in range(len(xpath_lists_movies[n])):
            all_dictionaries.append({'url': xpath_lists_urls_movies[t],
                                    'title': xpath_lists_movies[n][t],
                                    'cast': xpath_lists_movies_cast_matches[t]})
        
        
    return all_dictionaries

In [145]:
all_dictionaries_movies = make_dict_movies()

In [149]:
import json
with open('test_movies.jl', 'w') as f:
    f.write('\n'.join(map(json.dumps, all_dictionaries_movies)))

Пример с обходом страниц

In [ ]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [ ]:
# dataclass

class MovieItem(scrapy.Item):
    title = scrapy.Field()
    rating = scrapy.Field()
    summary = scrapy.Field()
    genre = scrapy.Field()
    runtime = scrapy.Field()
    directors = scrapy.Field()
    writers = scrapy.Field()
    cast = scrapy.Field()

class ActorItem(scrapy.Item):
    actor_name = scrapy.Field()
    character = scrapy.Field()

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://imdb.com"
    start_urls = ['https://www.imdb.com/chart/top',]
   
    def parse(self, response):
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        for row in table_rows:
            rating = row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            rel_url = row.xpath("td[@class='titleColumn']/a/@href").extract_first().strip()
            row_url = self.base_url + rel_url
            yield scrapy.Request(row_url, callback=self.parseOneMovie, meta={'rating' : rating})
    
    def parseOneMovie(self, response):
        item = MovieItem()
        item["rating"] = response.meta["rating"][0]
        item['title'] = response.xpath('.//*/div[@class="title_wrapper"]/h1/text()').extract_first().strip()
        item["summary"] = response.xpath(".//*/div[@class='summary_text']/text()").extract_first().strip()
        item['directors'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract_first().strip()
        item['writers'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract()[1].strip()

        item["cast"] = list()

        for cast in response.xpath(".//table[@class='cast_list']/tr")[1:]:
            actor = ActorItem()
            
            actor["actor_name"] = cast.xpath("./td[2]/a/text()").extract_first().strip()
            actor["character"] = cast.xpath("./td[@class='character']/a/text()").extract_first()
            
            item["cast"].append(actor)

        return item

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

In [ ]:
DEBUG_URL = 'http://www.imdb.com/title/tt0019254/'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [ ]:
selector.xpath(".//table[@class='cast_list']/tr")